# Lezione 4 - Funzioni Avanzate in Python

Questo notebook esplora tecniche avanzate di programmazione funzionale in Python, costruendo sulle conoscenze di base delle funzioni già acquisite.

## 1. Decoratori in Python

I decoratori sono funzioni che modificano il comportamento di altre funzioni. Permettono di "avvolgere" una funzione esistente con codice aggiuntivo, estendendo le sue funzionalità senza modificarne il codice interno.

### Concetti chiave:
- I decoratori prendono una funzione come input e ne restituiscono una nuova
- Vengono applicati usando la sintassi `@nome_decoratore` sopra la definizione della funzione
- Sono utili per aggiungere funzionalità comuni come logging, timing, autenticazione, ecc.

In [ ]:
# Esempio di decoratore semplice
def decoratore_semplice(funzione):
    def wrapper():
        print("Prima della chiamata alla funzione")
        funzione()
        print("Dopo la chiamata alla funzione")
    return wrapper

@decoratore_semplice
def saluta():
    print("Ciao!")
    
# Chiamare la funzione decorata
saluta()

# Questo è equivalente a:
# saluta = decoratore_semplice(saluta)

In [ ]:
# Decoratore per funzioni con parametri
def decoratore_con_parametri(funzione):
    def wrapper(*args, **kwargs):
        print(f"Chiamata alla funzione {funzione.__name__} con argomenti: {args}, {kwargs}")
        risultato = funzione(*args, **kwargs)
        print(f"La funzione ha restituito: {risultato}")
        return risultato
    return wrapper

@decoratore_con_parametri
def addizione(a, b):
    return a + b

addizione(5, 3)

In [ ]:
# Uso di functools.wraps per mantenere i metadata della funzione originale
from functools import wraps

def mio_decoratore(funzione):
    @wraps(funzione)  # Preserva informazioni come __name__, __doc__, ecc.
    def wrapper(*args, **kwargs):
        print("Eseguo decorazione")
        return funzione(*args, **kwargs)
    return wrapper

@mio_decoratore
def funzione_esempio(x):
    """Questa è una funzione di esempio."""
    return x * 2

print(funzione_esempio.__name__)  # Stampa "funzione_esempio" invece di "wrapper"
print(funzione_esempio.__doc__)   # Preserva la docstring

## 2. Closures e Variabili Nonlocal

Una closure è una funzione che ricorda i valori nell'ambito in cui è stata creata, anche quando quella funzione viene eseguita al di fuori di quell'ambito. Le closure sono alla base di molti pattern avanzati in Python.

In [ ]:
# Esempio di closure
def crea_moltiplicatore(x):
    def moltiplica(y):
        return x * y  # Utilizza 'x' dalla funzione genitore
    return moltiplica

# Creiamo due funzioni specializzate
raddoppia = crea_moltiplicatore(2)
triplica = crea_moltiplicatore(3)

print(raddoppia(5))  # 10
print(triplica(5))   # 15

In [ ]:
# Utilizzo della parola chiave nonlocal
def crea_contatore():
    conteggio = 0
    
    def incrementa():
        nonlocal conteggio  # Permette di modificare la variabile nell'ambito esterno
        conteggio += 1
        return conteggio
    
    return incrementa

contatore = crea_contatore()
print(contatore())  # 1
print(contatore())  # 2
print(contatore())  # 3

# Ogni contatore mantiene il proprio stato
contatore2 = crea_contatore()
print(contatore2())  # 1
print(contatore())   # 4 (il primo contatore continua da dove era rimasto)

## 3. Ricorsione e Memoization

La ricorsione è una tecnica in cui una funzione chiama se stessa per risolvere problemi complessi. La memoization è una tecnica di ottimizzazione che memorizza i risultati di chiamate di funzione costose e riutilizza i risultati quando gli stessi input si ripresentano.

In [ ]:
# Funzione ricorsiva per il calcolo del fattoriale
def fattoriale(n):
    if n <= 1:
        return 1
    else:
        return n * fattoriale(n - 1)

print(fattoriale(5))  # 120

In [ ]:
# Sequenza di Fibonacci con ricorsione (inefficiente)
def fibonacci(n):
    if n <= 1:
        return n
    else:
        return fibonacci(n-1) + fibonacci(n-2)

# Per numeri piccoli funziona bene
print([fibonacci(i) for i in range(10)])

# Ma diventa molto lento per numeri più grandi
import time
start = time.time()
print(f"fibonacci(30) = {fibonacci(30)}")
print(f"Tempo: {time.time() - start:.2f} secondi")

In [ ]:
# Ottimizzazione con memoization manuale
def fibonacci_memo():
    memo = {}
    
    def calcola(n):
        if n in memo:
            return memo[n]
        if n <= 1:
            result = n
        else:
            result = calcola(n-1) + calcola(n-2)
        memo[n] = result
        return result
    
    return calcola

fib = fibonacci_memo()

# Ora è molto più veloce
start = time.time()
print(f"fibonacci(30) = {fib(30)}")
print(f"Tempo: {time.time() - start:.2f} secondi")

# Proviamo anche con numeri più grandi
start = time.time()
print(f"fibonacci(100) = {fib(100)}")
print(f"Tempo: {time.time() - start:.2f} secondi")

## 4. Annotazioni di Funzione e Type Hints

Le annotazioni di funzione e i type hints permettono di specificare i tipi di parametri e valori di ritorno delle funzioni. Sono utili per la documentazione, il controllo statico dei tipi e il completamento automatico negli IDE.

In [ ]:
# Funzione con annotazioni di tipo
def saluta(nome: str) -> str:
    return f"Ciao, {nome}!"

print(saluta("Paolo"))

# Le annotazioni sono disponibili come attributi della funzione
print(saluta.__annotations__)

In [ ]:
# Annotazioni più complesse con il modulo typing
from typing import List, Dict, Tuple, Optional, Union

def elabora_dati(
    valori: List[int], 
    opzioni: Dict[str, bool] = None, 
    configurazione: Optional[Tuple[int, str]] = None
) -> Union[int, float]:
    """
    Questa funzione riceve una lista di interi e restituisce un numero.
    
    Args:
        valori: Una lista di numeri da elaborare
        opzioni: Dizionario di opzioni per l'elaborazione
        configurazione: Configurazione aggiuntiva opzionale
        
    Returns:
        Il risultato dell'elaborazione, come intero o float
    """
    # Il codice qui sarebbe l'implementazione effettiva
    return sum(valori)

# L'IDE può usare queste informazioni di tipo per suggerimenti
risultato = elabora_dati([1, 2, 3, 4], {"normalizza": True})
print(risultato)

## 5. Funzioni Parziali con functools

Le funzioni parziali permettono di "preimpostare" alcuni argomenti di una funzione, creando una nuova funzione con meno parametri. Sono utili quando si vuole creare versioni specializzate di funzioni esistenti.

In [ ]:
from functools import partial

# Funzione base con molti parametri
def potenza(base, esponente):
    return base ** esponente

# Creazione di funzioni specializzate
quadrato = partial(potenza, esponente=2)
cubo = partial(potenza, esponente=3)

print(quadrato(5))  # 25
print(cubo(5))      # 125

# Altro esempio con più parametri
def formato_messaggio(messaggio, prefisso, suffisso):
    return f"{prefisso} {messaggio} {suffisso}"

# Creazione di formattatori specializzati
formatta_errore = partial(formato_messaggio, prefisso="ERRORE:", suffisso="!")
formatta_info = partial(formato_messaggio, prefisso="Info:", suffisso=".")

print(formatta_errore("File non trovato"))
print(formatta_info("Operazione completata"))

## 6. Caching di Funzioni con lru_cache

Il decoratore `lru_cache` memorizza automaticamente i risultati delle chiamate di funzione, evitando calcoli ripetuti. È particolarmente utile per funzioni ricorsive o operazioni costose.

In [ ]:
from functools import lru_cache
import time

# Funzione Fibonacci con cache
@lru_cache(maxsize=None)  # maxsize=None indica cache illimitata
def fibonacci_cached(n):
    if n <= 1:
        return n
    return fibonacci_cached(n-1) + fibonacci_cached(n-2)

# Misuriamo le prestazioni
start = time.time()
print(f"fibonacci_cached(35) = {fibonacci_cached(35)}")
print(f"Tempo: {time.time() - start:.4f} secondi")

# Una seconda chiamata è praticamente istantanea grazie alla cache
start = time.time()
print(f"fibonacci_cached(35) = {fibonacci_cached(35)}")  # Risultato da cache
print(f"Tempo: {time.time() - start:.8f} secondi")

# Possiamo vedere le statistiche della cache
print(f"Info cache: {fibonacci_cached.cache_info()}")

In [ ]:
# Esempio pratico: cache per richieste API
@lru_cache(maxsize=100)
def recupera_dati(id_utente):
    print(f"Recupero dati per l'utente {id_utente}...")
    time.sleep(1)  # Simuliamo una richiesta di rete
    return {"id": id_utente, "nome": f"Utente {id_utente}"}

# Prima chiamata (lenta)
start = time.time()
print(recupera_dati(42))
print(f"Tempo: {time.time() - start:.2f} secondi")

# Seconda chiamata allo stesso utente (veloce)
start = time.time()
print(recupera_dati(42))
print(f"Tempo: {time.time() - start:.2f} secondi")

# Chiamata per un utente diverso (lenta di nuovo)
start = time.time()
print(recupera_dati(43))
print(f"Tempo: {time.time() - start:.2f} secondi")

## 7. Funzioni Generatrici e yield

Le funzioni generatrici utilizzano la parola chiave `yield` per produrre una sequenza di valori senza crearla interamente in memoria. Sono estremamente efficienti per lavorare con grandi insiemi di dati.

In [ ]:
# Funzione generatrice semplice
def contatore(max):
    i = 0
    while i < max:
        yield i
        i += 1

# Utilizzo del generatore
for numero in contatore(5):
    print(numero)

# I generatori sono iterabili una sola volta
gen = contatore(3)
print(list(gen))  # [0, 1, 2]
print(list(gen))  # [] (già esaurito)

In [ ]:
# Generatori per processare grandi file in modo efficiente
def leggi_file_a_blocchi(nome_file, dimensione_blocco=1024):
    with open(nome_file, 'r') as f:
        while True:
            blocco = f.read(dimensione_blocco)
            if not blocco:
                break
            yield blocco

# Esempio di utilizzo (senza un file reale)
def simula_lettura_file():
    # Generatore per simulare la lettura di un file grande
    for i in range(3):
        yield f"Questo è il blocco {i} di dati dal file.\n"
        yield f"Contiene multiple righe di testo.\n"
        yield f"Fine blocco {i}.\n"

# Conteggio delle righe usando il generatore
conteggio_righe = 0
for blocco in simula_lettura_file():
    # Contiamo le righe in ciascun blocco
    conteggio_righe += blocco.count('\n')
    
print(f"Il file contiene {conteggio_righe} righe")

## 8. Context Managers con contextlib

I context manager gestiscono l'acquisizione e il rilascio di risorse. Il decoratore `@contextmanager` insieme a `yield` permette di creare facilmente nuovi context manager.

In [ ]:
# Context manager tradizionale usando la clausola with
with open('temp.txt', 'w') as f:
    f.write('Hello, world!')

# Ora creiamo un nostro context manager
from contextlib import contextmanager
import time

@contextmanager
def timer():
    start = time.time()
    try:
        yield  # Questo è dove verrà eseguito il corpo del blocco with
    finally:
        end = time.time()
        print(f"Tempo di esecuzione: {end - start:.4f} secondi")

# Utilizzo del nostro context manager
with timer():
    # Codice da misurare
    result = sum(i**2 for i in range(1000000))
    print(f"Risultato: {result}")

In [ ]:
# Context manager per gestione delle risorse
@contextmanager
def gestione_connessione(host):
    print(f"Connessione a {host}...")
    # In un caso reale, qui si aprirebbe realmente una connessione
    conn = {"host": host, "connected": True}
    try:
        yield conn  # Forniamo la connessione al blocco with
    except Exception as e:
        print(f"Errore durante l'utilizzo della connessione: {e}")
        # Gestione dell'errore
    finally:
        print(f"Chiusura connessione a {host}")
        # In un caso reale, qui si chiuderebbe la connessione
        conn["connected"] = False

# Utilizzo
with gestione_connessione("server.example.com") as conn:
    print(f"Connesso a {conn['host']}")
    # Utilizzo della connessione
    print("Eseguiamo alcune operazioni...")

print("\nAltro esempio con gestione degli errori:")
try:
    with gestione_connessione("backup.example.com") as conn:
        print(f"Connesso a {conn['host']}")
        # Simuliamo un errore
        raise ValueError("Errore durante il trasferimento dati")
except ValueError:
    print("Errore gestito a livello superiore")

## 9. Funzioni di Ordine Superiore

Le funzioni di ordine superiore sono funzioni che operano su altre funzioni, prendendole come argomenti o restituendole come risultato. In Python, le funzioni di ordine superiore native più comuni sono `map()`, `filter()` e `reduce()`.

In [ ]:
# Esempio di map, filter e reduce
from functools import reduce

# Una lista di numeri
numeri = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

# Map - applica una funzione a ogni elemento
quadrati = list(map(lambda x: x**2, numeri))
print("Map (quadrati):", quadrati)

# Filter - seleziona elementi in base a una condizione
pari = list(filter(lambda x: x % 2 == 0, numeri))
print("Filter (pari):", pari)

# Reduce - aggrega gli elementi in base a una funzione
somma = reduce(lambda acc, x: acc + x, numeri, 0)
print("Reduce (somma):", somma)

# Prodotto di tutti i numeri
prodotto = reduce(lambda acc, x: acc * x, numeri, 1)
print("Reduce (prodotto):", prodotto)

In [ ]:
# Funzioni che restituiscono funzioni
def crea_validatore(min_valore, max_valore):
    def validatore(x):
        return min_valore <= x <= max_valore
    return validatore

# Creazione di validatori specifici
valida_voto = crea_validatore(0, 10)
valida_percentuale = crea_validatore(0, 100)

# Uso dei validatori
print(valida_voto(7))       # True
print(valida_voto(11))      # False
print(valida_percentuale(75))  # True
print(valida_percentuale(110)) # False

# Combinare funzioni di ordine superiore
dati = [
    {"nome": "Alice", "voto": 95},
    {"nome": "Bob", "voto": 75},
    {"nome": "Charlie", "voto": 83},
    {"nome": "Diana", "voto": 62},
    {"nome": "Eva", "voto": 88}
]

# Trovare gli studenti con voto superiore a 80
voti_alti = list(filter(lambda studente: studente["voto"] > 80, dati))
nomi_voti_alti = list(map(lambda studente: studente["nome"], voti_alti))
print("Studenti con voto > 80:", nomi_voti_alti)

## 10. Composizione di Funzioni

La composizione di funzioni è una tecnica in cui l'output di una funzione diventa l'input di un'altra. Permette di creare pipe di trasformazioni di dati, un concetto fondamentale nella programmazione funzionale.

In [ ]:
# Funzioni di base per le trasformazioni
def raddoppia(x):
    return x * 2

def incrementa(x):
    return x + 1

def quadrato(x):
    return x ** 2

# Composizione manuale
valore = 3
risultato = quadrato(incrementa(raddoppia(valore)))
print(f"Composizione manuale: {valore} -> {risultato}")

In [ ]:
# Creazione di una utility per la composizione di funzioni
def componi(*funzioni):
    """Compone una serie di funzioni da destra a sinistra."""
    def funzione_composta(x):
        risultato = x
        # Applica le funzioni in ordine inverso (da destra a sinistra)
        for f in reversed(funzioni):
            risultato = f(risultato)
        return risultato
    return funzione_composta

# Utilizzo della composizione
f = componi(quadrato, incrementa, raddoppia)
print(f"Composizione con utility: 3 -> {f(3)}")  # (3*2 + 1)^2 = 49

# Altra utility per composizione da sinistra a destra (stile pipe)
def pipe(valore, *funzioni):
    """Fa passare un valore attraverso una sequenza di funzioni."""
    risultato = valore
    for f in funzioni:
        risultato = f(risultato)
    return risultato

print(f"Composizione con pipe: 3 -> {pipe(3, raddoppia, incrementa, quadrato)}")  # (3*2 + 1)^2 = 49

In [ ]:
# Esempio pratico: pipeline di elaborazione dati
dati = [
    {"nome": "Marco", "voto": 85, "corso": "Python"},
    {"nome": "Giulia", "voto": 92, "corso": "JavaScript"},
    {"nome": "Paolo", "voto": 78, "corso": "Python"},
    {"nome": "Anna", "voto": 95, "corso": "Python"},
    {"nome": "Luca", "voto": 70, "corso": "JavaScript"}
]

# Funzioni di trasformazione
def filtra_corso_python(studenti):
    return [s for s in studenti if s["corso"] == "Python"]

def ordina_per_voto(studenti):
    return sorted(studenti, key=lambda s: s["voto"], reverse=True)

def estrai_nomi(studenti):
    return [s["nome"] for s in studenti]

def formatta_lista(nomi):
    return ", ".join(nomi)

# Applicazione della pipeline
risultato = pipe(
    dati,
    filtra_corso_python,
    ordina_per_voto,
    estrai_nomi,
    formatta_lista
)

print("Studenti Python ordinati per voto (dal più alto):", risultato)

## Conclusione

In questo notebook abbiamo esplorato concetti avanzati di funzioni in Python:

- **Decoratori**: Per modificare o estendere il comportamento delle funzioni
- **Closures**: Per creare funzioni che ricordano il loro ambiente
- **Ricorsione e memoization**: Per affrontare problemi complessi in modo elegante ed efficiente
- **Annotazioni e type hints**: Per documentare e verificare i tipi dei parametri e valori di ritorno
- **Funzioni parziali**: Per creare varianti specializzate di funzioni esistenti
- **Caching con lru_cache**: Per ottimizzare funzioni costose
- **Generatori**: Per lavorare in modo efficiente con grandi sequenze di dati
- **Context managers**: Per gestire l'acquisizione e il rilascio di risorse
- **Funzioni di ordine superiore**: Per operare su altre funzioni
- **Composizione di funzioni**: Per costruire pipeline di elaborazione

Questi strumenti offrono potenti meccanismi per scrivere codice Python più elegante, modulare e performante.